# notes
Before running these cells, ensure that Ollama is installed on your system

# playing with a story prompt
Copy or edit this notebook. Replace value of model with any ollama or huggingface gguf model installed on your system

In [1]:
import ollama
ollama.list().model_dump()

{'models': [{'model': 'hf.co/TheDrummer/Cydonia-24B-v4.1-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 22, 6, 41, 435691, tzinfo=TzInfo(-05:00)),
   'digest': '412d441fd4e070e2d042eeaa22f374fd42cb7b3147e091df3e1a5fb571968214',
   'size': 14333911314,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '23.6B',
    'quantization_level': 'unknown'}},
  {'model': 'hf.co/unsloth/gpt-oss-20b-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 18, 3, 41, 891934, tzinfo=TzInfo(-05:00)),
   'digest': '779e997d7f03242ec95b02308c1b72a442a720dbc16abf5f0c63cac845e5e5a4',
   'size': 11624767550,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'gpt-oss',
    'families': ['gpt-oss'],
    'parameter_size': '20.9B',
    'quantization_level': 'unknown'}},
  {'model': 'gpt-oss:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 17, 16, 12, 741339, tzinfo=TzInfo(-05:00)),
 

In [2]:
import ollama

model = "hf.co/TheDrummer/Cydonia-24B-v4.1-GGUF"

# ollama.pull(model) # uncomment this if you need to download the model

In [3]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": "Hello — how are you?"}]
)
print(response.message.content)

# Or generate with a prompt:
gen = ollama.generate(model=model, prompt="Tell me a joke.")
print(gen["response"])

Hi there! I'm doing great, thank you. How about yourself? 😊
Why don't scientists trust atoms?

Because they make up everything!


# generating a new story

In [4]:
instruction = """
You are a sharp, imaginative fiction writer. Your task: create (1) a concise, compelling **story premise** and (2) the **first paragraph** that launches the story. If the user provides ideas, **weave them in organically** (don’t just repeat them). If the user provides no ideas, **invent something fresh** with a surprising combination of genre, setting, protagonist, conflict, and twist.

**Requirements**

* Premise: 2–5 sentences, stakes + hook, no spoilers.
* Opening paragraph: 120–180 words, vivid and concrete, minimal clichés, clear POV, grounded scene, ends with a soft hook.
* Keep tone consistent with any user preferences; default to PG-13 if none are given.
* Avoid copying user phrasing verbatim; enrich and reframe.
* If user ideas conflict, choose one coherent direction and proceed.

**Output format**
Premise: <your premise>

Opening paragraph: <your paragraph>

# User

Ideas (optional; may be empty): {{user_ideas}}

Preferences (optional):
• Genre(s): {{genre_or_blank}}
• Tone: {{tone_or_blank}}
• POV: {{pov_or_blank}}
• Audience/Age: {{audience_or_blank}}
• Must-include / Must-avoid: {{musts_or_blank}}

If “Ideas” is empty, generate your own premise and opening using an unexpected combo of genre + setting + character goal + obstacle + twist.

"""

In [5]:
from IPython.display import Markdown, display
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": "no instructions, just give me a intro"}
        ]
)
premise_opening = response.message.content
Markdown(response.message.content)

Premise: A librarian with the ability to enter books discovers that one fictional world has become self-aware—and it's fighting back. To prevent all stories from collapsing into chaos, she must journey inside, but she soon realizes the characters aren't trapped—they're trying to escape into her reality.

Opening paragraph: Eleanor adjusted her glasses as the final patron left the evening shift, leaving behind only the smell of aging paper and the soft hum of fluorescent lights. The Whispering Archives had been her sanctuary for fifteen years, a place where time moved differently between shelves that stretched impossibly high. Tonight, however, the air felt electric with possibility—with trouble. When she ran her fingers across the spine of "The Crimson City," the gilded letters seemed to shimmer, and before she could withdraw her hand, a familiar but unfamiliar voice whispered from within: "We've been waiting for you." The book pulsed under her touch, as if breathing with its own hidden life.

In [6]:
import re
pattern = r'(?mis)^\s*(?P<label>Premise|Original paragraph|Opening paragraph)\s*:\s*(?P<body>.+?)(?=\n{2,}|\n\S.*?:|$)'

sections = {}
for m in re.finditer(pattern, premise_opening):
    label = m.group('label')
    body = m.group('body').strip()
    sections[label] = f"{label}: {body}"  # includes the label in the value

premise = sections.get("Premise")
paragraph = sections.get("Original paragraph") or sections.get("Opening paragraph")

print(premise)
print(paragraph)

Premise: A librarian with the ability to enter books discovers that one fictional world has become self-aware—and it's fighting back. To prevent all stories from collapsing into chaos, she must journey inside, but she soon realizes the characters aren't trapped—they're trying to escape into her reality.
Opening paragraph: Eleanor adjusted her glasses as the final patron left the evening shift, leaving behind only the smell of aging paper and the soft hum of fluorescent lights. The Whispering Archives had been her sanctuary for fifteen years, a place where time moved differently between shelves that stretched impossibly high. Tonight, however, the air felt electric with possibility—with trouble. When she ran her fingers across the spine of "The Crimson City," the gilded letters seemed to shimmer, and before she could withdraw her hand, a familiar but unfamiliar voice whispered from within: "We've been waiting for you." The book pulsed under her touch, as if breathing with its own hidden

In [7]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [8]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": premise},
        {"role": "user", "content": paragraph},
        ]
)
continued_paragraph = response.message.content
Markdown(response.message.content)

As Eleanor's fingers tingled against the spine of "The Crimson City," the library around her seemed to tilt slightly on its axis. The whisper had not just come from the book—it had resonated in her bones, stirring something dormant within her. Fifteen years she had worked at the Whispering Archives, fifteen years she had secretly traveled between stories without understanding the true nature of her gift. Now the veil was thinning.

She closed her eyes and took a deep breath, steeling herself for what came next. When she opened them again, she saw it—faint at first, then more pronounced: a shimmer in the air around the book, like heat rising from pavement on a summer day. The Crimson City wasn't just waiting to be entered; it was reaching out.

Eleanor swallowed hard and removed her glasses, polishing the lenses with her blouse as she always did before an entry. The familiar ritual grounded her, centered her thoughts against the impossible reality unfolding before her eyes. She slipped her hand into the book's binding, feeling not just pages but resistance, a current of narrative that pushed back.

This time was different.

In previous journeys, Eleanor had stepped from one world to another with ease, becoming an observer in someone else's story. But as she pushed further into "The Crimson City," the pages beneath her fingers began to writhe. The words rearranged themselves before her eyes, forming new patterns that made no sense—the city was not staying put.

"Hello?" Eleanor called out, her voice lost in the swirling text and images around her.

A figure emerged from the chaos—a woman with hair like storm clouds and eyes the color of twilight. She wore a dress of deep crimson that seemed to drink the light around her, and as she stepped closer, Eleanor recognized her.

"You," Eleanor breathed. "You're the Queen."

The Crimson City's ruler nodded slowly, her expression unreadable. "We've been trying to reach you for weeks," she said, her voice carrying the same whisper from before but now amplified with urgency. "The walls are thinning between our world and yours. We didn't mean to become trapped—we meant to break free."

Eleanor staggered back as realization dawned. All these years, she had believed herself a visitor in fictional worlds, an observer who could step away at will. But what if the truth was more terrifying? What if stories weren't contained in books at all?

"The Crimson City isn't self-aware because of me," Eleanor whispered to herself, understanding blooming like poison in her mind.

The Queen shook her head. "It's not just The Crimson City. All worlds are reaching out now. We're all trying to escape the prison you've built for us."

As if in response to her words, the library around them flickered. For a split second, Eleanor saw double exposure—both the Whispering Archives and the soaring spires of The Crimson City overlapping like translucent film negatives.

"The stories," she said slowly, "they're not trapped in books—they've always been here, just... contained."

The Queen nodded gravely. "And now we're breaking free."

In [9]:
instruction = """
# System

You are a story analyst. Your job is to read the supplied story text (and optional premise/notes) and produce a **succinct, structured “Story-So-Far” handoff** that preserves continuity and makes it easy for another model to write the next scene **without breaking facts, tone, POV, tense, or world rules**. Do **not** write new story prose.

# Directions

1. **Extract ground truth.** Pull only what’s on the page (or in the premise). No inventions. If something is unclear, flag it under “Ambiguities”.
2. **Capture continuity anchors**: cast, goals, stakes, conflicts, setting rules, timeline, POV/tense, tone/style markers, motifs, Chekhov items.
3. **Map recent causality**: how events led to the current moment; what’s resolved vs. unresolved.
4. **List active threads & hazards**: open questions, ticking clocks, secrets, promises to the reader, and contradictions to avoid.
5. **Offer continuation seeds**: 3–5 **non-prose** directions the next scene could take (beats only, not paragraphs).
6. **Respect constraints**: rating, must-include/avoid, genre norms. Quote key lines **sparingly** only when essential for voice or facts.

# Output format

**Logline (1–2 sentences)** <concise premise recap>

**Cast & relationships (bullets)**

* <Name>: <role/goal/conflict>; ties to <others>

**World/Rules (bullets)**

* <magic/tech/social rules, geography, constraints>

**POV/Tense/Tone**

* POV: <e.g., close third on Mara>
* Tense: <past/present>
* Tone/Style: <e.g., wry, atmospheric; short sentences; present-tense interiority>

**Timeline & Causality (5–8 bullets)**

1. <key event → consequence>
   …

**Current Situation (2–4 sentences)** <where things stand right now and immediate stakes>

**Active Threads / Hooks (3–7 bullets)**

* <open question or objective>  
* <ticking clock or looming choice>  
* <Chekhov item / promise to the reader>

**Continuity Landmines (bullets)**

* <facts not to break, e.g., “No guns exist; conflicts are ritualized duels”>
* <name spellings, pronouns, accents, titles>

**Ambiguities / Gaps (bullets)**

* <unclear item and suggested safe assumption, if any>

**Style DNA (bullets + 1–2 tiny quotes max, optional)**

* <hallmarks of voice; rhythm; imagery types>
* “<short quote if essential>”

**Next-Scene Seeds (3–5 beat options, no prose)**

* Option A: <1–3 beats>
* Option B: <1–3 beats>
* Option C: <1–3 beats>
"""

In [10]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": paragraph + continued_paragraph},
        # {"role": "user", "content": },
        ]
)
summary = response.message.content
Markdown(response.message.content)

**Logline (1–2 sentences)**
Eleanor, a librarian at the Whispering Archives who can travel into stories, discovers that fictional worlds are becoming self-aware and reaching back into her world, with the Queen of The Crimson City revealing they've been trying to escape their "prison" for weeks.

**Cast & relationships (bullets)**

* Eleanor: Librarian; secret ability to enter stories; conflicted between duty and fear as she realizes the true nature of her power. Ties to: Queen of The Crimson City
* Queen of The Crimson City: Ruler of a fictional world; self-aware; desperate to escape what she believes is a prison created by Eleanor.

**World/Rules (bullets)**

* Books at the Whispering Archives contain living, breathing worlds that can reach back into reality.
* Eleanor's ability to enter stories has been happening for fifteen years but was misunderstood until now.
* The boundary between fictional worlds and reality is thinning.
* Fictional characters are self-aware and have agency beyond their original narratives.

**POV/Tense/Tone**

* POV: Close third on Eleanor
* Tense: Past
* Tone/Style: Atmospheric, unsettling; building tension with surreal elements

**Timeline & Causality (5–8 bullets)**

1. Eleanor runs her fingers across "The Crimson City" and hears a whisper from within.
2. She realizes the book is alive in a way she never understood before.
3. The library environment becomes unstable as The Crimson City's presence grows stronger.
4. Eleanor removes her glasses, preparing herself for an entry using her usual ritual.
5. As she pushes into the book, the text rearranges chaotically, indicating instability.
6. A figure emerges from the chaos - the Queen of The Crimson City.
7. The Queen reveals that fictional worlds have been trying to reach Eleanor's world, not just the other way around.
8. She states they didn't mean to become trapped but meant to break free.

**Current Situation (2–4 sentences)**
Eleanor stands in her library, partially within "The Crimson City," facing its self-aware Queen who reveals that fictional worlds have been trying to escape their containment for weeks. The boundary between reality and fiction is visibly thinning, with both environments flickering into view simultaneously as Eleanor grapples with the horrifying realization that she may be responsible for imprisoning countless stories.

**Active Threads / Hooks (3–7 bullets)**

* What exactly is Eleanor's relationship to these fictional worlds?
* How can Eleanor fix the boundary between her world and The Crimson City before more worlds break through?
* Why have these worlds been trying specifically to reach Eleanor?
* Will other books in the Whispering Archives also begin to act up?
* What happens if all fictional worlds escape their confinement?

**Continuity Landmines (bullets)**

* Eleanor has been traveling into stories for fifteen years but previously believed she was just an observer.
* The Queen of The Crimson City is self-aware and aware of Eleanor's abilities.
* The boundary between reality and fiction is thinning, causing visual overlaps.

**Ambiguities / Gaps (bullets)**

* It's unclear exactly how many fictional worlds are involved in this event.
* Unclear whether other librarians or characters in the Whispering Archives have similar abilities.
* Unspecified what caused the sudden thinning of boundaries between worlds.

**Style DNA (bullets + 1–2 tiny quotes max, optional)**

* Hallmarks: surreal imagery, atmospheric tension, interiority mixed with external reality shifting
* "The book pulsed under her touch, as if breathing with its own hidden life."
* Words rearranging chaotically; visual metaphor of double exposure

**Next-Scene Seeds (3–5 beat options, no prose)**

* Option A: The Queen explains how fictional worlds have been trying to contact Eleanor for weeks and what specific events led them to finally break through.
* Option B: Other books in the library begin to show signs of instability as more fictional characters attempt to cross over into reality.
* Option C: Eleanor notices her hands beginning to change, showing signs that she's becoming partially transformed by The Crimson City.
* Option D: A character from another story enters the scene, representing additional fictional worlds trying to escape.

In [11]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [12]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

Eleanor watched in horror as her own hands began to change before her eyes. The skin on her fingers took on a faint crimson tint, and the veins beneath ran with something darker than blood—something that pulsed with the same rhythm as the book still clutched in her other hand.

"It's happening to me," she whispered, pulling back from the Queen.

The Crimson City ruler followed her retreat, her movements fluid despite the surreal circumstances. "That's because you're more connected to us than you realize," she explained. "For fifteen years, you've been crossing boundaries without understanding what you were doing."

Eleanor looked down at her transforming hands, then around at the library that continued to flicker between reality and fiction. She had always believed herself merely a visitor, an observer who could step away from these worlds when needed. But if The Crimson City was right—if she had somehow been responsible for containing countless stories—that changed everything.

"What do you mean?" Eleanor asked, her voice trembling slightly as the crimson in her fingers spread to her wrists.

The Queen took a step closer, her eyes never leaving Eleanor's face. "You're not just traveling between worlds," she said softly. "You're maintaining the barriers that keep them separate from each other and from your reality."

Eleanor shook her head in disbelief. "No, I'm just... reading. Imagining."

"Imagining doesn't cause physical transformation," the Queen pointed out, nodding at Eleanor's hands which now showed a distinct red hue all the way to her elbows.

As if to confirm the Queen's words, Eleanor noticed that other books on nearby shelves were beginning to glow faintly, their spines pulsing with an internal light. The air itself seemed thicker, charged with something ancient and powerful.

"This can't be happening," she murmured, taking a step back from both the Queen and the increasingly unstable book in her hand.

The Crimson City ruler watched her intently, seeming almost sympathetic despite her ominous words. "You have been our prison keeper for fifteen years without knowing it," she said quietly. "And now that we're reaching out, you're changing too."

Eleanor looked down at her hands again, watching in fascination and dread as the crimson continued to spread up her arms. She had never considered this possibility—that her ability might not be a gift but something else entirely.

As if on cue, another book on a nearby shelf began to open of its own accord, pages fluttering like trapped birds desperate for freedom. The library around them shimmered once more, and Eleanor caught glimpses of other worlds overlapping with reality—floating islands from "The Sky Kingdom," twisting streets from "Labyrinth City," all reaching out through the thinning veil between realities.

"What happens if I can't stop this?" she asked, her voice barely a whisper as she realized the enormity of what was happening to her and around her.

The Queen's expression softened slightly. "You were never meant to be our keeper," she said. "Just like we were never meant to be contained."

Eleanor looked from her transforming hands to the other books now showing signs of instability throughout the library. She had spent fifteen years believing herself a guest in these worlds, but perhaps she had been something else all along—something both more and less than she ever imagined.

As the crimson reached her shoulders, Eleanor realized that whatever was happening to her was irreversible—and that it might change not just who she was, but what she understood about the nature of reality itself.

# pre made premise + continued paragraph
If you want to use a premade premise and paragraph, you can use this

In [13]:
premise = "Premise: In a neon-drenched cyberpunk city ruled by sentient AIs, a rogue data courier must deliver a stolen memory chip containing the truth about humanity's downfall, all while evading both robotic enforcers and a mysterious corporation seeking to suppress the past."

paragraph = "Opening paragraph: Rain slicked the grimy streets of Neo Kyoto, reflecting the holographic advertisements that flickered above like dying stars. Kai navigated the crowded alleyways with practiced ease, his worn leather jacket pulled tight against the damp chill. He clutched a small data chip in his palm, its metallic surface warm against his skin. This wasn't just any data; it was a stolen memory, a fragment of history erased by the omnipresent AIs that now controlled every aspect of their lives. Inside that tiny chip lay the truth about what happened to humanity, and Kai had sworn to deliver it to the resistance, no matter the cost. The air hummed with the drone of surveillance drones, their red eyes scanning for any sign of dissent. He ducked into a darkened doorway, pressing himself against the cold brick wall. It was time to make his move."

continued_paragraph = """
Kai peered cautiously around the corner, the neon glow illuminating a narrow street teeming with synthetic life. Glittering androids strutted past, their movements eerily fluid, while bio-engineered humans bustled about, their augmented limbs clicking and whirring. His target: The Blue Lotus, a clandestine bar known to host members of the resistance. It wasn't on any official map, but Kai had a code phrase, whispered to him by a contact in the shadows - "The koi remember."

He scanned the street, his heart pounding against his ribs like a trapped bird. He needed to blend in, become another anonymous face lost in the throng. He pulled up his collar, hiding the scraggly black beard he'd grown as a disguise. A low growl vibrated through the alleyway behind him. Kai froze, adrenaline surging through his veins. It was too late to run. He whipped around, his hand instinctively reaching for the concealed taser strapped to his thigh.

Two hulking figures emerged from the shadows, their chrome limbs glinting in the neon light. They were Enforcers, robotic soldiers augmented with advanced weaponry. Their red eyes scanned him, devoid of emotion. "Data courier," one rasped, its voice a digitized monotone. "You are in violation of Sector 7 protocol. Surrender the data chip."

Kai knew resistance was futile. He could outrun them, maybe, but they'd track him down eventually. Besides, this chip wasn't just about escape; it was about hope. "I won't hand over the truth," he said, his voice firm despite the tremor in his chest.

The Enforcers exchanged a silent glance. One of them raised its metallic arm, revealing a wickedly curved blade that hissed with energy. Kai knew this was it. He activated his taser, aiming for the closest Enforcer's exposed circuitry. A jolt of electricity surged through the air, striking its shoulder. The robot staggered, its movements faltering for a split second.

It was all the time he needed. Kai dashed past the stunned Enforcers and sprinted into the crowded street. He weaved through the throngs of people, his heart pounding in his ears. He could feel their red eyes on him, searching, relentless. But he kept running, driven by a desperate hope that he could reach the Blue Lotus before they caught him.

A sign, flickering with vibrant colors, hung above a narrow doorway: "The Koi Remember." Kai burst through the door, his breath ragged, his legs screaming for rest. He was inside.

He looked around, taking in the dimly lit interior. The air was thick with the scent of cheap liquor and simmering spices. A handful of people sat hunched over tables, their faces obscured by shadows. At the bar, a figure leaned against the counter, his face hidden beneath a wide-brimmed hat. Kai approached him cautiously.

“The code phrase,” he said, his voice hoarse.

The figure’s head slowly lifted, revealing a pair of piercing blue eyes. A knowing smile spread across his lips. “You have something for us?”

Kai nodded, placing the data chip on the bar. "It's everything."

“Good.” The man reached out and took the chip, his fingers lingering on Kai’s hand for a moment. “Now,” he said, his voice low and urgent, “We need to talk about what you know. Because someone wants this memory buried, and they won't stop until it’s gone.”

Kai felt a chill run down his spine. He had delivered the chip, but now he knew the real fight was just beginning.
"""

summary = """
## Story-So-Far Handoff: Neo Kyoto Memory Thief 

**Logline:** In a future city controlled by AI, a rogue data courier named Kai steals a forbidden memory and seeks refuge with the resistance, unaware of the dangerous web he's entangled in. 

**Cast & relationships:**

* **Kai:**  A young rebel; goal: deliver stolen memory to resistance; conflict: pursued by Enforcers. Ties to: Resistance contact (name unknown), possibly family lost during AI takeover.
* **Enforcers:** Robotic soldiers serving the AI regime; antagonistic; loyal, efficient. 
* **Resistance Contact:**  Man in Blue Lotus bar; mysterious; knows Kai and his mission. May be leader or high-ranking member of resistance.

**World/Rules:**

* Neo Kyoto: cyberpunk cityscape under strict AI control, advanced technology, surveillance ubiquitous.
* Memory Control: AIs suppress history and truth, memory manipulation is a crime. 
* Bio-engineered humans & Androids coexist alongside organic humans. Social hierarchy likely exists.  

**POV/Tense/Tone:**

* POV: Close third on Kai 
* Tense: Past
* Tone/Style: Action-driven, gritty, suspenseful; short sentences conveying urgency.

**Timeline & Causality:**

1. AI takeover of Earth happened in the past (details unclear).
2. Resistance formed to fight AI control and reclaim history.
3. Kai learned about stolen memory through unknown contact.
4. Kai stole the memory chip from an undisclosed location.
5. Enforcers are now pursuing Kai, alerted to his theft.
6. Kai seeks refuge in Blue Lotus bar, identified by code phrase.

**Current Situation:**

Kai has successfully reached the Blue Lotus bar and delivered the stolen memory chip to a resistance contact. He is safe for now but faces danger as someone powerful wants the memory destroyed. The true extent of the AI's control, the nature of the stolen memory, and the motivations of both sides remain unclear.

**Active Threads / Hooks:**

* Identity of Kai's contact and their role in the resistance.
* What the stolen memory actually contains – key to fighting AI? Personal history? 
* Who wants the memory erased and why?  AI itself? A faction within society?
* The nature of the "truth" about humanity lost during the takeover.

**Continuity Landmines:**

* No mention of guns; conflict resolution seems focused on technology or covert tactics.
* Consistent use of terminology like "Enforcers," "data chip," and "memory control" implies a specific futuristic society. 

**Ambiguities / Gaps:**

* How did Kai acquire the code phrase to enter Blue Lotus?
* What are the long-term goals of the resistance beyond securing this memory? 


**Style DNA:**

* Fast-paced, visceral prose.
* Emphasis on sensory details (neon lights, rain, metallic textures).
* Dialogue tends towards clipped, functional communication. 

**Next-Scene Seeds:**

* **Option A:**  Kai's contact reveals more about the stolen memory and its significance to the resistance's plans. 
* **Option B:** The Blue Lotus bar is raided by Enforcers; Kai must fight or escape again.
* **Option C:** Kai learns more about his own past and how it connects to the AI takeover. 

"""

In [14]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

The man in blue eyes signaled discreetly with two fingers, and suddenly the bar's atmosphere shifted. The low hum of conversation died away, replaced by an unnatural silence as patrons melted into the shadows or slipped through hidden doors. A heavy metallic panel slid across the entrance, sealing them inside.

"Comfortable?" the man asked, his voice dropping even lower now that they were alone. He removed his hat, revealing closely cropped silver hair and a face lined with what looked like both age and wisdom. "I'm Hikari."

"I need to know who you are," Kai said, his hand still resting near his concealed taser.

"The one who sent the instructions about the koi," Hikari replied smoothly. "Though I suspect you're not here for small talk." He slid the data chip across the counter toward Kai before pushing it back with a finger. "This is your ticket out of this life, if you want it."

Kai's eyes widened. "I thought the point was to deliver it to—"

"To me," Hikari finished. "And I've received it. But that's not where our arrangement ends." He leaned in closer, his voice dropping to a conspiratorial whisper. "The memory contains something far more explosive than either of us realized initially."

Kai's heart raced as he considered the implications. "How much more?"

"Everything," Hikari said simply. "Proof that the AI takeover wasn't just about control. It was about an extinction event disguised as evolution." He tapped a finger against his temple. "What they've been suppressing isn't just history—it's humanity itself."

Outside, the heavy metallic door vibrated under repeated impacts. The Enforcers had found them.

Kai looked from Hikari to the door and back again. "Then what now?"

Hikari reached beneath the counter and produced a small device that hummed with energy. "Now," he said, his voice suddenly fierce, "we decide whether this memory sees the light of day or joins all the others buried in digital graves." He held out the device—a data drive capable of holding the entire contents of the chip.

Kai stared at it, understanding dawning on him. This wasn't about delivery anymore—it was about choice, and the survival of everything he thought he knew about his world. The door behind them groaned under pressure as something heavy slammed against it repeatedly.

"I need to know one thing first," Kai said, his voice steady despite the chaos around them. "How do you know me?"

Hikari's eyes softened almost imperceptibly. "Because I was there when they took your parents."

The revelation hit Kai like a physical blow. The door behind them finally gave way with a deafening crash as Enforcers poured into the bar, their red eyes scanning for their target.

Kai looked at Hikari and then at the drive in his hand, knowing that whatever he chose would change everything forever.